# 잠재 디리클레 할당(LDA)

LDA를 수행할 떄 문서 집합에서 토픽이 몇개가 존재할지 가정하는 것은 사용자가 한다.

LDA 역시 DTM 또는 TF-IDF 행렬을 입력으로 받는데, 이를 통해 LDA는 단어의 순서를 신경쓰지 않는다는 것을 알 수 있다.

LDA는 문서들로부터 '나는 이 문서를 작성하기 위해서 이런 주제들을 넣을 것이고, 이런 단어들을 넣을거야' 라는 가정을 염두해두고 진행한다.

예를들어 각각의 문서가
1. 문서에 사용할 단어의 개수 N을 정한다.
2. 문서에 사용할 토픽의 혼합을 확률 분포에 기반하여 결정한다.
3. 문서에 사용할 단어를 토픽 분포에서 토픽 T를 확률적으로 고르고, 선택한 토픽 T에서 단어의 출현 확률 분포에 기반해 문서에 사용할 단어를 룬다.

고 가정하고 LDA는 토픽을 뽑아내기 위해 위의 과정을 역으로 추적한다.

## LDA의 수행과정

1. 사용자는 알고리즘에게 토픽의 개수 k개를 알려준다.

LDA는 토픽의 개수 k를 입력받으면, k개의 토픽이 M개의 전체 문서에 걸쳐 분포되어 있다고 가정한다.

2. 모든 단어를 k개 중 하나의 토픽에 할당한다.

위 작업을 통해 각 문서는 토픽을 가지며, 토픽은 단어 분포를 가진다. 물론 랜덤이기 때문에 결과는 전부 틀리다.

3. 모든 문서의 모든 단어에 대해 다음 사항 반복진행.
- 어떤 문서의 각 단어 w는 자신은 잘못된 토픽에 할당되어 있지만, 다른 단어들은 전부 올바른 토픽에 할당되어 있다고 가정한다. 이데 따라 단어 w는 아래의 두가지 기준에 따라 토픽이 재할당된다.

    - p(topic t | document d): 문서 d의 단어들 중 토픽 t에 해당하는 단어들의 비율
    - p(word w | topic t): 각 토픽들 t에서 해당 단어 w의 분포
    
4. 수렴할 때 까지 진행.

LSA vs LDA

- LSA: DTM을 차원숙소하여 근접 단어들을 토픽으로 묶는다.
- LDA: 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하여 토픽을 추출한다.

# 실습 01: gensim 사용

gensim: Word2Vec 지원, 토픽 모델링 지원하는 패키지

## 정수 인코딩과 단어 집합 만들기

LSA와 전처리 과정을 같고 tokenized_doc로 토큰화 했다고 하자.

In [3]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import warnings
warnings.filterwarnings(action='ignore')

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

#전처리
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

# NLTK로부터 불용어를 받아온다.
import nltk
nltk.download('stopwords')

# 불용어를 제거
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jihyeonbin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flintstone', 'chewables', 'bake', 'timmons']


In [4]:
# 토큰화 완료
print(tokenized_doc[:5])

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object


각 단어에 정수 인코딩을 하는 동시에, 각 뉴스에서의 단어의 빈도수 기록한다.

각 단어를 (word_id, word_frequeny) 형태로 변경하자. 
- word_id: 단어가 정수 인코딩 된 값
- word_frequency: 해당 뉴스에서의 해당 단어의 빈도수

정수 인코딩: 단어를 빈도수 순으로 정렬한 단어 집합을 만들고 빈도수가 높은 순서대로 차례로 낮은 숫자부터 정수 부여

In [6]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

#전체 뉴스에 대해 정수 인코딩 수행하고, 두번째 뉴스 출력

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


위 출력결과 중 (66,2)는 정수 인코딩이 66으로 할당된 단어가 두번째 뉴스에서 두번 등장했다는 의미이다.

단어가 정수 인코딩 되기 전 어떤 단어였는지 확인도 가능하다.

In [7]:
print(dictionary[66])

faith


In [8]:
len(dictionary)

64281

총 65284개의 단어가 학습되었다

## LDA 모델 훈련시키기

기존의 뉴스 데이터가 총 20개의 카테고리를 가지고 있으므로 토픽의 개수를 20으로 하여 LDA 모델을 학습시켜보자.

In [9]:
import gensim
NUM_TOPICS = 20 # 20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.017*"israel" + 0.015*"jews" + 0.012*"turkish" + 0.010*"israeli"')
(1, '0.013*"colormap" + 0.010*"width" + 0.008*"pixmap" + 0.008*"height"')
(2, '0.016*"jesus" + 0.009*"christian" + 0.009*"bible" + 0.008*"church"')
(3, '0.008*"time" + 0.007*"president" + 0.006*"year" + 0.005*"space"')
(4, '0.026*"henrik" + 0.017*"norton" + 0.011*"allocation" + 0.009*"cross"')
(5, '0.020*"file" + 0.016*"windows" + 0.013*"program" + 0.010*"window"')
(6, '0.019*"period" + 0.011*"power" + 0.011*"pittsburgh" + 0.009*"play"')
(7, '0.012*"drive" + 0.011*"like" + 0.008*"system" + 0.008*"card"')
(8, '0.014*"andrew" + 0.013*"doug" + 0.012*"symbol" + 0.007*"friday"')
(9, '0.019*"people" + 0.018*"would" + 0.011*"like" + 0.011*"know"')
(10, '0.013*"available" + 0.010*"information" + 0.008*"also" + 0.008*"mail"')
(11, '0.009*"true" + 0.009*"believe" + 0.008*"argument" + 0.008*"evidence"')
(12, '0.010*"ground" + 0.008*"wire" + 0.008*"water" + 0.008*"current"')
(13, '0.041*"armenian" + 0.028*"armenians" + 0.014*

각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도를 보여준다. 즉, 20개의 토픽에서 각 토픽에 대한 기여도를 볼 수 있다. 

passess는 알고리즘의 동작 횟수를 말하는데 알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 정해준다.

num_words = 4: 총 4개의 단어만 출력

## LDA 시각화

In [12]:
pip install pyLDAvis

  Using cached pandas-1.5.3-cp38-cp38-macosx_10_9_x86_64.whl (11.9 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.0
    Uninstalling pandas-1.2.0:
      Successfully uninstalled pandas-1.2.0
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install pandas==1.2

/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  Using cached pandas-1.2.0-cp38-cp38-macosx_10_9_x86_64.whl (10.5 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.2
    Uninstalling pandas-1.3.2:
      Successfully uninstalled pandas-1.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.11 requires pandas>=1.3.0, but you have pandas 1.2.0 which is incompatible.
pyldavis 3.4.0 requires pandas>=1.3.4, but you have pandas 1.2.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


판다스 높으면 시각화에 오류가 생긴다.

In [20]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


- 좌측의 원: 각각 20개의 토픽을 나타낸다.
- 각 원과의 거리: 각 토픽들이 서로 얼마나 다른지.
- 만약 두개의 원이 겹친다면 유사한 토픽.
- LDA 시각화에서는 토픽의 번호가 1부터 시작한다

## 문서 별 토픽 분포 보기

위 시각화 결과는 토픽 별 단어 분포이다.

문서 별 토픽 분포를 확인해보자.

각 문서의 토픽 분포는 훈련된 LDA 모델인 ldamodel[]에 전체 데이터가 정수 인코딩 된 결과를 넣은 후 확인 가능하다.

상위 5개 문서에 대한 토픽을 확인해보자.

In [22]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(0, 0.29123423), (2, 0.054613512), (8, 0.09846291), (9, 0.4753993), (19, 0.06797002)]
1 번째 문서의 topic 비율은 [(9, 0.5395148), (11, 0.28814065), (12, 0.14996399)]
2 번째 문서의 topic 비율은 [(0, 0.3765783), (1, 0.016922697), (9, 0.57717127), (17, 0.016398111)]
3 번째 문서의 topic 비율은 [(3, 0.19951566), (4, 0.01535701), (7, 0.16350228), (9, 0.14557658), (11, 0.060901497), (15, 0.016745824), (16, 0.3882147)]
4 번째 문서의 topic 비율은 [(3, 0.39254716), (5, 0.092319794), (18, 0.06449187), (19, 0.42099798)]


/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(숫자,확률)로 나타나는 결과는 각각 토픽 번호와 토픽이 해당 문서에서 차지하는 분포도를 나타낸다.

예를 들어 (0,0.2912)는 0번 토픽이 29%의 분포를 가진다는 것을 알 수 있다.

데이터프레임으로 출력해보자.

In [23]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)


topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,9.0,0.4754,"[(0, 0.29121318), (2, 0.054601748), (8, 0.0984..."
1,1,9.0,0.5396,"[(9, 0.53955877), (11, 0.2880958), (12, 0.1499..."
2,2,9.0,0.5772,"[(0, 0.37658405), (1, 0.016922703), (9, 0.5771..."
3,3,16.0,0.3883,"[(3, 0.19969477), (4, 0.015356995), (7, 0.1640..."
4,4,19.0,0.4209,"[(3, 0.39258894), (5, 0.092326894), (18, 0.064..."
5,5,2.0,0.6392,"[(2, 0.6392157), (9, 0.31985402)]"
6,6,7.0,0.5980,"[(5, 0.10054704), (7, 0.59802383), (11, 0.0342..."
7,7,9.0,0.4879,"[(0, 0.1006638), (3, 0.3574871), (9, 0.4878755..."
8,8,3.0,0.2546,"[(3, 0.25455418), (7, 0.24977902), (9, 0.22558..."
9,9,7.0,0.2991,"[(0, 0.030273702), (3, 0.26224503), (7, 0.2991..."


# 실습 02: 사이킷런을 활용한 실습

## 데이터 생성
15년간 발생된 뉴스 기사 제목을 모아놓은 영어 데이터 다운받아보자.

https://www.kaggle.com/datasets/therohk/million-headlines?resource=download

In [26]:
import pandas as pd
import urllib.request
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation


data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)
print('뉴스 제목 개수 :',len(data))

/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


뉴스 제목 개수 : 1244184


In [27]:
print(data.head(5))

   publish_date                                      headline_text
0      20030219  aba decides against community broadcasting lic...
1      20030219     act fire witnesses must be aware of defamation
2      20030219     a g calls for infrastructure protection summit
3      20030219           air nz staff in aust strike for pay rise
4      20030219      air nz strike to affect australian travellers


/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


- publish_date: 뉴스가 나온 날짜
- headline_text: 뉴스 기사 제목

In [28]:
text = data[['headline_text']]
text.head(5)

/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


## 텍스트 전처리

불용어 제거, 표제어 추출, 길이가 짧은 단어 제거

In [29]:
#단어 토큰화
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
print(text.head(5))

                                       headline_text
0  [aba, decides, against, community, broadcastin...
1  [act, fire, witnesses, must, be, aware, of, de...
2  [a, g, calls, for, infrastructure, protection,...
3  [air, nz, staff, in, aust, strike, for, pay, r...
4  [air, nz, strike, to, affect, australian, trav...


/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
#불용어 제거
stop_words = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop_words)])

/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
#표제어 추출: 3인칭 단수를 1인칭으로 바꾸고 과거 현재형 동사를 현재형으로 바꾼다.

import nltk
nltk.download('wordnet')
    
    
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
print(text.head(5))

/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jihyeonbin/nltk_data...


                                       headline_text
0       [aba, decide, community, broadcast, licence]
1      [act, fire, witness, must, aware, defamation]
2      [g, call, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


In [34]:
#길이가 3 이하인 단어 제거
import nltk
nltk.download('stopwords')

tokenized_doc = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 3])
print(tokenized_doc[:5])

/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jihyeonbin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0       [decide, community, broadcast, licence]
1      [fire, witness, must, aware, defamation]
2    [call, infrastructure, protection, summit]
3                   [staff, aust, strike, rise]
4      [strike, affect, australian, travellers]
Name: headline_text, dtype: object


## TF-IDF 행렬 만들기

In [35]:
# 역토큰화 (토큰화 작업을 되돌림)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

# 다시 text['headline_text']에 재저장
text['headline_text'] = detokenized_doc

/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
#1000개로 TF-IDF 행렬 만들어보자
# 상위 1,000개의 단어를 보존 
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000)
X = vectorizer.fit_transform(text['headline_text'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TF-IDF 행렬의 크기 : (1244184, 1000)


## 토픽 모델링

In [37]:
lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)

/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
lda_top = lda_model.fit_transform(X)

/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
# 단어 집합. 1,000개의 단어가 저장됨.
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

Topic 1: [('australia', 20556.0), ('sydney', 11219.29), ('melbourne', 8765.73), ('kill', 6646.06), ('court', 6004.12)]
Topic 2: [('coronavirus', 41719.62), ('covid', 28960.68), ('government', 9793.89), ('change', 7576.98), ('home', 7457.74)]
Topic 3: [('south', 7102.98), ('death', 6825.39), ('speak', 5402.31), ('care', 4521.48), ('interview', 4058.71)]
Topic 4: [('donald', 8536.49), ('restrictions', 6456.4), ('world', 6320.61), ('state', 6087.5), ('water', 4219.67)]
Topic 5: [('vaccine', 8040.87), ('open', 6915.17), ('coast', 5990.55), ('warn', 5472.84), ('morrison', 5247.93)]
Topic 6: [('trump', 14878.13), ('charge', 7717.96), ('health', 6836.95), ('murder', 6663.45), ('house', 6624.13)]
Topic 7: [('australian', 13885.88), ('queensland', 13373.53), ('record', 9037.88), ('test', 7713.9), ('help', 5922.05)]
Topic 8: [('case', 13146.83), ('police', 11143.1), ('live', 7528.03), ('border', 6855.54), ('tasmania', 5664.64)]
Topic 9: [('victoria', 11777.5), ('school', 6009.78), ('attack', 550

/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


아래 코드는 별개이다.

Sklearn LDA는 fit과 transform 함수를 제공하는데 fit 호출까지는 토픽별 단어들의 분포를 반환해주고 transform까지 하면 문서별 토픽들의 분포까지 반환해준다.

아래 코드에서 토픽별 단어들의 분포까지만 반환하고 우리에게 어떻게 시각화 되는지 지켜보자

In [41]:
lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)

lda_model.fit(X) #X는 행렬

print(lda_model.components_.shape)
print(lda_model.components_)

/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(10, 1000)
[[1.00005784e-01 1.00000401e-01 1.00000837e-01 ... 1.00008966e-01
  1.00003001e-01 1.00003989e-01]
 [1.00001271e-01 1.00000170e-01 1.00000702e-01 ... 1.00009027e-01
  1.00004822e-01 6.11677308e+02]
 [1.00001893e-01 1.00000747e-01 5.29631843e+02 ... 1.00004665e-01
  1.00003828e-01 1.00003730e-01]
 ...
 [1.00006258e-01 1.00000570e-01 1.00000388e-01 ... 1.00006276e-01
  1.00001666e-01 1.00008566e-01]
 [1.00000280e-01 1.00000135e-01 1.00001609e-01 ... 1.00003391e-01
  1.00001003e-01 1.00006488e-01]
 [1.03272231e+02 1.00000349e-01 1.00001881e-01 ... 1.00005116e-01
  1.00004106e-01 1.00006425e-01]]


10개의 토픽, 1000개의 feature 나타난다.

토픽별로 어떤 단어들이 분포하는지 확인해보자

In [42]:
def display_topic_words(lda_model,feature_names,num_top_words):
    for topic_idx, topic in enumerate(lda_model.components_):
        print('\nTopic #', topic_idx +1)
        
        #Topic별 1000개의 feaure중 높은 값 순으로 정렬 후 index 반환
        #argsort는 디폴트가 오름차순이라 내림차순으로 변경
        
        topic_word_idx = topic.argsort()[::-1]
        top_idx = topic_word_idx[:num_top_words]
        
        #CountVectorizer 함수 할당시킨 객체에 get_feature_names()로 벡터화시킨 feautres 확인 가능
        #이 벡터화된 features는 숫자-알파벳 순으로 정렬
        # 문자열.join 함수로 특정 문자열 사이에 끼고 합쳐주자
        
        feautre_concat = '+'.join([str(feature_names[i]) + '*' + str(round(topic[i],1)) for i in top_idx])
        print(feautre_concat)

/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
count_vect = CountVectorizer(max_df = 0.95, max_features = )

In [48]:
from sklearn.feature_extraction.text import CountVectorizer

feautre_names = count_vect.get_feature_names()
display_topic_words(lda_model, feature_names, 15)

/Users/jihyeonbin/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TypeError: get_feature_names() missing 1 required positional argument: 'self'